In [15]:
import sys, os
# os.add_dll_directory(r"C:\msys64\ucrt64\bin")
# sys.path.insert(0, r"C:\Users\fabia\Personal\Uni\Shared\scicomp\schaf3\build\mechsystem")
from pathlib import Path

import importlib.util

p = (Path.cwd().parent / "build" / "mechsystem").resolve()
sys.path.insert(0, str(p))

print("spec:", importlib.util.find_spec("mass_spring"))
import mass_spring
print("imported:", mass_spring)

from pythreejs import *

FileNotFoundError: [Errno 2] No such file or directory

In [10]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

mA = mss.add (Mass(1, (1,0,0)))
mB = mss.add (Mass(2, (2,0,0)))
mC = mss.add (Mass(3, (4,0,0)))
f1 = mss.add (Fix( (0,0,0)) )
mss.add (Spring(1, 200000, (f1, mA)))
mss.add (Spring(1, 100000, (mA, mB)))
mss.add (Spring(2, 100, (mB, mC)))

2

In [11]:
# Add distance constraints instead of (or in addition to) springs
# Replace the springs above with rigid distance constraints
mss.add(DistanceConstraint(1.0, (f1, mA)))  # rigid distance 1.0
mss.add(DistanceConstraint(1.0, (mA, mB)))  # rigid distance 1.0
mss.add(DistanceConstraint(2.0, (mB, mC)))  # rigid distance 2.0


2

In [12]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [13]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [14]:
from time import sleep
for i in range(10000):
    mss.simulate (0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

KeyboardInterrupt: 